# Langchain Exploration - Quickstart

In this notebook I explore the basics of Lanchain + PDF retrieval.

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [2]:
print(llm.invoke("What's that Monty Python and the Holy Grail quote about the swallow?"))


The famous Monty Python and the Holy Grail quote about the swallow is:

"Never underestimate the power of a good swallow."

This quote is spoken by King Arthur in the film, as he is trying to convince his knights that they should not underestimate the abilities of the peasant woman who has just been accused of stealing the Holy Grail. The knights are skeptical and dismissive of her, but King Arthur argues that she may have powers beyond their comprehension:

"She's got legs, sir! She's got legs!"

The quote has become a catchphrase and cultural reference point, often used humorously or ironically to refer to something that seems unexpectedly powerful or impressive.


## PDF Loading

Following https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-207.pdf")
pages = loader.load_and_split()

In [4]:
pages[0]

Document(page_content='NIST Special Publication 800 -207 \n \nZero Trust Architecture  \n \n \nScott Rose  \nOliver Borchert  \nStu Mitchell \nSean Connelly  \n \n \n \n \nThis publication is available free of charge from:  \nhttps://doi.org/10.6028/ NIST.SP.800- 207 \n \n \n C  O  M  P  U  T  E  R      S  E  C  U  R  I  T  Y', metadata={'source': 'https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-207.pdf', 'page': 0})

In [5]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(pages, OllamaEmbeddings())

In [6]:
faiss_index.similarity_search("Multicloud")

[Document(page_content='NIST  SP 800-207  ZERO TRUST ARCHITECTURE  \n   \n43 \nThis publication is available free of charge from: https:// doi.org/10.6028/ NIST.SP.800 -207 Program. (The White House, Washington, DC), OMB Memorandum M -\n19-03, December 10, 2018. Available at https://www.whitehouse.gov/wp-\ncontent/uploads/2018/12/M -19-03.pdf  \n[M-19-17] Office of Management and Budget (2019) Enabling Mission Delivery \nthrough Improved Identity, Credential, and Access Management. (The \nWhite House, Washington, DC), OMB Memorandum M -19-17, May 21, \n2019. Available at https://www.whitehouse.gov/wp-\ncontent/uploads/2019/05/M -19-17.pdf  \n[M-19-19] Office of Management and Budget (2019) Update on Data Center \nOptimization Initiative (DCOI). (The White House, Washington, DC), OMB Memorand um M -19-19, June 25, 2019. Available at \nhttps://datacenters.cio.gov/assets/files/m_19_19.pdf  \n[M-19-26] Office of Management and Budget (2019) Update to the Trusted Internet \nConnections ( TI

In [7]:
retriever = faiss_index.as_retriever()

In [8]:
retriever.invoke("References")

[Document(page_content='NIST  SP 800-207  ZERO TRUST ARCHITECTURE  \n   \n5 \nThis publication is available free of charge from: https:// doi.org/10.6028/ NIST.SP.800 -207 In the abstract model of access shown in Figure 1, a subject needs access to an enterprise \nresource. Access is granted through a policy decision point (PDP) and corresponding policy enforcement point (PEP).\n3  \n \n \nFigure 1: Zero Trust Access  \nThe system must ensure that the subject is authentic and the request is valid. The PDP/PEP passes proper judgment to allow the subject to access the resource. This implies that zero trust applies to two basic areas: authentication and authorization. What is the level of confidence about the subject’s identity for this unique request? Is access to the resource al lowable given the \nlevel of confidence in the subject’s identity? Does the device used for the request have the proper security posture? Are there other factors that should be considered and that change the con

## Prompt with Retreival

Following https://medium.com/scrapehero/building-qa-bot-with-ollama-local-llm-platform-c11d1e1f3035

In [9]:
from langchain import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don’t know the answer, just say that you don’t know, don’t try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""

prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [10]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})

In [11]:
print(chain.invoke({"query": "Summarize the document."})['result'])

The document "NIST SP 800-207 Zero Trust Architecture" provides a framework for protecting enterprise resources and data by assuming that all access to resources is untrusted, regardless of the user's location or device. The zero trust architecture focuses on restricting resource access to only those with a legitimate need to access them, and granting access only to the minimum privileges required to perform the intended action. The document provides guidelines for implementing a zero trust architecture, including:

1. Understanding the basics of zero trust and its importance in protecting enterprise resources.
2. Identifying the key components of a zero trust architecture, including the policy engine, policy administrator, and the underlying infrastructure.
3. Discussing the potential threats to a zero trust architecture, such as unauthorized access, network disruptions, and denial-of-service attacks.
4. Providing recommendations for mitigating these threats, including proper configur